# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [19]:
from torchvision import transforms, datasets
import torch
from torch import optim
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd

import os
import json

from PIL import Image
from torchsummary import summary
import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['font.size'] = 18

from timeit import default_timer as timer

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

trained_model = False
batch_size = 16

train_on_gpu = torch.cuda.is_available()
train_on_gpu

True

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

In [20]:
data_dir = '/home/ubuntu/git/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [21]:
train_n = 0
valid_n = 0

for d in os.listdir(train_dir):
    train_n += len(os.listdir(train_dir + f'/{d}'))
    
for d in os.listdir(valid_dir):
    valid_n += len(os.listdir(valid_dir + f'/{d}'))
train_n, valid_n

# TODO: Define your transforms for the training and validation sets
image_transforms = {
        'train': transforms.Compose([
                 transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                 transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                 transforms.ToPILImage(),
                 transforms.TenCrop(size=224),
                 transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops]))
        ]),
        'val': transforms.Compose([
                transforms.Resize(size=256),
                transforms.CenterCrop(size=224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }

batch_size = 16

# Datasets
data = {'train': datasets.ImageFolder(root=train_dir,
                                      transform=image_transforms['train']),
        'val': datasets.ImageFolder(root=valid_dir,
                                     transform=image_transforms['val'])
       }

dataloaders = {'train': DataLoader(data['train'], batch_size=batch_size),
              'val': DataLoader(data['val'], batch_size=batch_size)
              }

(6552, 818)

In [13]:
# from PIL import Image
# img = Image.open('../flower_data/train/99/image_07834.jpg')

# t =  transforms.Compose([
#              transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
#     transforms.ToPILImage(),
#              transforms.TenCrop(size=224),
#             transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),   
#         ])

# t(img)

In [22]:
len(dataloaders['train'].dataset.samples)
len(dataloaders['train'].dataset.classes)

validiter = iter(dataloaders['val'])
next(validiter)[0].shape

trainiter = iter(dataloaders['train'])
next(trainiter)[0].shape

6552

102

torch.Size([16, 3, 224, 224])

torch.Size([16, 10, 3, 224, 224])

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [23]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

list(cat_to_name.items())[:6]

[('21', 'fire lily'),
 ('3', 'canterbury bells'),
 ('45', 'bolero deep blue'),
 ('1', 'pink primrose'),
 ('34', 'mexican aster'),
 ('27', 'prince of wales feathers')]

In [24]:
class_to_idx = data['train'].class_to_idx
idx_to_name = {idx: cat_to_name[category] for category, idx in data['train'].class_to_idx.items()}
list(idx_to_name.items())[:6]

[(0, 'pink primrose'),
 (1, 'globe thistle'),
 (2, 'blanket flower'),
 (3, 'trumpet creeper'),
 (4, 'blackberry lily'),
 (5, 'snapdragon')]

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [25]:
model = models.vgg16(pretrained=True)

# Freeze training for all layers
for param in model.parameters():
    param.requires_grad = False
    
model.classifier[6]

Linear(in_features=4096, out_features=1000, bias=True)

In [26]:
# Classifier module
class Sequential(nn.Module):
    def __init__(self, n_inputs, n_classes, drop_prob=0.2):
        super(Sequential, self).__init__()
        # Fully connected layer
        self.fc1 = nn.Linear(n_inputs, int(n_inputs / 4))
        self.fc2 = nn.Linear(int(n_inputs / 4), int(n_inputs / 8))
        self.fc3 = nn.Linear(int(n_inputs / 8), n_classes)
        
        # Dropout
        self.dropout = nn.Dropout(p=drop_prob)
        
        # Output layer
        self.out = nn.LogSoftmax(dim = 1)
    
    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.out(x)
        return x

In [27]:
n_inputs = model.classifier[6].in_features
n_classes = len(dataloaders['train'].dataset.classes)
model.classifier[6] = nn.Linear(n_inputs, n_classes)
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=102, bias=True)
)

In [28]:
# TODO: Build and train your network
if train_on_gpu:
    model = model.to('cuda')
from torchsummary import summary
summary(model, input_size = (3, 224, 224), batch_size = 1024)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1       [1024, 64, 224, 224]           1,792
              ReLU-2       [1024, 64, 224, 224]               0
            Conv2d-3       [1024, 64, 224, 224]          36,928
              ReLU-4       [1024, 64, 224, 224]               0
         MaxPool2d-5       [1024, 64, 112, 112]               0
            Conv2d-6      [1024, 128, 112, 112]          73,856
              ReLU-7      [1024, 128, 112, 112]               0
            Conv2d-8      [1024, 128, 112, 112]         147,584
              ReLU-9      [1024, 128, 112, 112]               0
        MaxPool2d-10        [1024, 128, 56, 56]               0
           Conv2d-11        [1024, 256, 56, 56]         295,168
             ReLU-12        [1024, 256, 56, 56]               0
           Conv2d-13        [1024, 256, 56, 56]         590,080
             ReLU-14        [1024, 256,

In [29]:
for param in model.parameters():
    if param.requires_grad:
        print(param.shape)

torch.Size([102, 4096])
torch.Size([102])


In [30]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Total parameters: {total_params:,}. Trainable parameters: {trainable_params:,}.')

Total parameters: 134,678,438. Trainable parameters: 417,894.


In [53]:
n_epochs = 20
max_epochs_stop = 3

save_file_name = 'aug_try.pt'

In [ ]:
for ii, (data, target) in enumerate(train_loader):
            bs, ncrops, c, h, w = data.size()
            
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            # Fuse batch size and the number of crops
            output = model(data.view(-1, c, h, w))

            # Take average over the crops
            output = output.view(bs, ncrops, -1).mean(1)
            
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()

            # Calculate accuracy
            _, pred = torch.max(output, dim = 1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            train_acc += accuracy.item()

In [38]:
def train(model, criterion, optimizer, train_loader, valid_loader, save_file_name,
          max_epochs_stop=3, n_epochs=20):

    # Early stopping details
    epochs_no_improve = 0
    valid_loss_min = np.Inf
    
    # Number of epochs already trained
    try:
        print(f'Current training epochs: {model.epochs}.')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.')
        
    overall_start = timer()
    # Iterate through epochs
    for epoch in range(n_epochs):

        # keep track of training and validation loss
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        model.train()
        
        start = timer()
        # Training loop
        for ii, (data, target) in enumerate(train_loader):
            bs, crops, c, h, w= data.size()
            
            # Tensors on gpu
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            
            # Clear gradients
            optimizer.zero_grad()
            
            # Predicted outputs
            # outputs are not probabilities
            output = model(data.view(-1, c, h, w))
            output = output.view(bs, crops, -1).mean(1)
        
            # Loss and backpropagation
            loss = criterion(output, target)
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Track train loss
            train_loss += loss.item()

            # Calculate accuracy by finding max probability
            _, pred = torch.max(output, dim = 1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            train_acc += accuracy.item()
            
            # Track training
            print(
                f'Epoch: {epoch}\t{100 * ii / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed.', end='\r')
        
        # After training loops ends
        else:
            model.epochs += 1
            model.eval()
            # Don't need to keep track of gradients
            with torch.no_grad():
                
                # Validation loop
                for data, target in valid_loader:
                    # Tensors to gpu
                    if train_on_gpu:
                        data, target = data.cuda(), target.cuda()
                        
                    # Forward pass
                    output = model(data)
                    # Validation loss
                    loss = criterion(output, target)
                    valid_loss += loss.item()

                    # Calculate validation accuracy
                    _, pred = torch.max(output, dim = 1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
                    valid_acc += accuracy.item()

                # Calculate average losses
                train_loss = train_loss/len(train_loader)
                valid_loss = valid_loss/len(valid_loader)
                # Calculate average accuracy
                train_acc = train_acc/len(train_loader)
                valid_acc = valid_acc/len(valid_loader)

                # Print training and validation results
                print('\nEpoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
                    epoch, train_loss, valid_loss))
                print(
                    f'Training Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%')

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min - 0.01:
                    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                        valid_loss_min,
                        valid_loss))
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    print(f'{epochs_no_improve} epochs with no improvement.')
                    if epochs_no_improve >= max_epochs_stop:
                        print('Early Stopping')
                        total_time = timer() - overall_start
                        print(f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.')
                        break

In [40]:
model, _ = load_checkpoint('vgg16.pth', models.vgg16(pretrained=True))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

train(model, criterion, optimizer,
      dataloaders['train'], dataloaders['val'],
      save_file_name='aug.pt', n_epochs = 25)

134,678,438 total parameters.
417,894 total gradient parameters.
Model has been trained for 46 epochs.
Current training epochs: 46.
Epoch: 0	99.76% complete. 824.44 seconds elapsed.
Epoch: 0 	Training Loss: 17.197520 	Validation Loss: 5.448834
Training Accuracy: 5.64%	 Validation Accuracy: 46.75%
Validation loss decreased (inf --> 5.448834).  Saving model ...
Epoch: 1	99.76% complete. 824.78 seconds elapsed.
Epoch: 1 	Training Loss: 10.765235 	Validation Loss: 5.669633
Training Accuracy: 9.59%	 Validation Accuracy: 47.72%
1 epochs with no improvement.
Epoch: 2	99.76% complete. 823.12 seconds elapsed.
Epoch: 2 	Training Loss: 9.515354 	Validation Loss: 6.689503
Training Accuracy: 12.71%	 Validation Accuracy: 45.91%
2 epochs with no improvement.
Epoch: 3	99.76% complete. 823.24 seconds elapsed.
Epoch: 3 	Training Loss: 8.659069 	Validation Loss: 7.600951
Training Accuracy: 15.96%	 Validation Accuracy: 44.35%
3 epochs with no improvement.
Early Stopping
3399.44 total seconds elapsed. 849.

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [36]:
def save_checkpoint(model, optimizer, path, save_cpu=False):
    if save_cpu:
        model = model.to('cpu')
        path = path.split('.')[0] + '-cpu.pth'
    
    checkpoint = {
        'cat_to_name': cat_to_name,
        'class_to_idx': data['train'].class_to_idx,
        'idx_to_name': idx_to_name,
        'epochs': model.epochs,
        'classifier': model.classifier,
        'state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }
    
    torch.save(checkpoint, path)
    
save_checkpoint(model, optimizer, 'vgg16-aug.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [37]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath, model):
    # Make sure to set parameters as not trainable
    for param in model.parameters():
        param.requires_grad = False
        
    # Load in checkpoint
    checkpoint = torch.load(filepath)
    # Extract classifier
    model.classifier = checkpoint['classifier']
    model.cat_to_name = checkpoint['cat_to_name']
    model.class_to_idx = checkpoint['class_to_idx']
    model.idx_to_name = checkpoint['idx_to_name']
    model.epochs = checkpoint['epochs']
    
    # Load in the state dict
    model.load_state_dict(checkpoint['state_dict'])
    
    if train_on_gpu:
        model = model.to('cuda')
    
    optimizer = optim.Adam(model.parameters())
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f'{total_params:,} total parameters.')
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'{total_trainable_params:,} total gradient parameters.')
    print(f'Model has been trained for {model.epochs} epochs.')
    
    return model, optimizer

model, optimizer = load_checkpoint('vgg16-aug.pth', models.vgg16(pretrained=True))
summary(model)

134,678,438 total parameters.
417,894 total gradient parameters.
Model has been trained for 4 epochs.


TypeError: summary() missing 1 required positional argument: 'input_size'

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

img = Image.open('../flower_data/valid/10/image_07094.jpg')
plt.imshow(img)

In [ ]:
img = img.resize((256, 256))
width = 256
height = 256
new_width = 224
new_height = 224

left = (width - new_width)/2
top = (height - new_height)/2
right = (width + new_width)/2
bottom = (height + new_height)/2

img = img.crop((left, top, right, bottom))
plt.imshow(img)

In [ ]:
img = np.array(img).transpose((2, 0, 1)) / 256

In [ ]:
means = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
stds = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))

img = img - means
img = img / stds

torch.Tensor(img)

In [ ]:
def process_image(image_path):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    image = Image.open(image_path)
    # Resize
    img = image.resize((256, 256))
    
    # Center crop
    width = 256
    height = 256
    new_width = 224
    new_height = 224

    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2
    img = img.crop((left, top, right, bottom))
    
    # Convert to numpy, transpose color dimension and normalize
    img = np.array(img).transpose((2, 0, 1)) / 256
    
    # Standardization
    means = np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
    stds = np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))

    img = img - means
    img = img / stds

    img_tensor = torch.Tensor(img)
    
    return img_tensor
    # TODO: Process a PIL image for use in a PyTorch model

In [ ]:
img = process_image('../flower_data/valid/11/image_03125.jpg')

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax, image

In [ ]:
ax, image = imshow(img)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    img_tensor = process_image(image_path)
    
    if train_on_gpu:
        img_tensor = img_tensor.view(1, 3, 224, 224).cuda()
    else:
        img_tensor = img_tensor.view(1, 3, 224, 224)
    
    with torch.no_grad():
        model.eval()
        out = model(img_tensor)
        log_ps = torch.softmax(out, dim = 1)
        ps = torch.exp(out)
        topk, topclass = ps.topk(topk, dim = 1)

        top_classes = [model.idx_to_name[class_] for class_ in topclass.cpu().numpy()[0]]
        return topk.cpu().numpy()[0], top_classes

# TODO: Implement the code to predict the class from an image file

In [ ]:
ex_path = '../flower_data/train/20/image_04897.jpg'
k, classes = predict(ex_path, model)

In [ ]:
k, classes

In [ ]:
model.idx_to_name[model.class_to_idx['20']]

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes